<a href="https://colab.research.google.com/github/BakhturinaPolina/goodreads-romance-research/blob/main/Scraping_Additional_Metadata_from_Goodreads_Book_Pages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell 1: Install Dependencies and Imports

In [5]:
# Install required packages (quiet to avoid verbose output)
!pip install beautifulsoup4 requests pandas selenium tqdm lxml webdriver-manager geckodriver-autoinstaller credentials --quiet

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from tqdm import tqdm
import json
import os
from datetime import datetime, timedelta

print("Cell 1: Setup complete.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cell 1: Setup complete.


# Cell 2: Cookie and Login Functions

In [6]:
def load_cookies(driver, auto_login=False):
    """
    Load cookies to simulate logged-in state or perform auto-login.
    """
    print("\n=== Starting cookie/login process ===")

    # Complete list of 13 cookies
    cookies = [
        {
            'name': '_session_id2',
            'value': '6c4ac9d7944645498873dcfaf76b294f',
            'domain': 'www.goodreads.com',
            'path': '/',
            'secure': True,
            'expiry': int((datetime.now() + timedelta(days=365)).timestamp())  # 1 year from now
        },
        {
            'name': 'at-main',
            'value': 'Atza|IwEBIP_WVjU2wDt5XXIwnYO37c_HBxHHpvejaw457NkT7-DHU7flVrkMM02XEH0cYeYRpEYlSuc7aN4cr-0ME-ruM28LUkrC7ODW8WKTpaqeH-sCuNVv1z9YM7xBS-Z4T0jhtLlPe8xzqTMozRae_ZNoxsqvpLNQxyDu0Woeei-Ip3E_PYxuZeikLbIDzpe17BVcrNocfqj4fl5KszGyF7ExHAvfVTGVBH_UeX5rQbSY6ZOCTHiIoca2U5sZfChiCoUH3wk',
            'domain': '.goodreads.com',
            'path': '/',
            'secure': True,
            'httpOnly': True,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        },
        {
            'name': 'ccsid',
            'value': '724-3321647-0772806',
            'domain': 'www.goodreads.com',
            'path': '/',
            'secure': False,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        },
        {
            'name': 'csm-hit',
            'value': 'tb:s-2R1WY81WFQ4QG5VKQVVG|1754352395507&t:1754352395508',
            'domain': 'www.goodreads.com',
            'path': '/',
            'secure': False,
            'expiry': int((datetime.now() + timedelta(days=365)).timestamp())
        },
        {
            'name': 'lc-main',
            'value': 'en_US',
            'domain': '.goodreads.com',
            'path': '/',
            'secure': False,
            'httpOnly': True,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        },
        {
            'name': 'locale',
            'value': 'en',
            'domain': 'www.goodreads.com',
            'path': '/',
            'secure': False
        },
        {
            'name': 'logged_out_browsing_page_count',
            'value': '1',
            'domain': 'www.goodreads.com',
            'path': '/',
            'secure': False,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        },
        {
            'name': 'sess-at-main',
            'value': 'U8NDSc1MTsE8derMKmAGMZ+Uq9chMayYrfiC+B46wCE=',
            'domain': '.goodreads.com',
            'path': '/',
            'secure': True,
            'httpOnly': True,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        },
        {
            'name': 'session-id',
            'value': '140-3588248-4268607',
            'domain': '.goodreads.com',
            'path': '/',
            'secure': False,
            'httpOnly': True,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        },
        {
            'name': 'session-id-time',
            'value': '2385072442l',
            'domain': '.goodreads.com',
            'path': '/',
            'secure': False,
            'httpOnly': True,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        },
        {
            'name': 'session-token',
            'value': '0+8P54n7E6d+0GGShiBALFOEgtTQdUdna1ExcAeRw3Ul16HkHsMc2W2ZpSYd+dn2CmsRT4KTntQt8WF4Of+YP1EitZ4QW4VQMG/hg3NoH61WK00ztRFxR6GkLoQiygwTiqkHExpG3pEjipb2/x256UoQDqyJcqAdFLyXukWtVKwfCFNsWzxJZMP/gmSX/Ml1mSdPMmkb9yJ5gb+ugF6z5a1F2Hr01Tt1Ynz77AY8fV5BunhvuaXJYoMJjEKHvWoLmurVyyWT/YZ71mOlJJXAtphVwQmemG3C27hNxcCL3cC3x1N6iA36seE5LzPJmOkxNjbGZ5EnO3s+IVXsenDzWijTZZz3dAu9eDn03BYbLEF0fsf7cW4b3g==',
            'domain': '.goodreads.com',
            'path': '/',
            'secure': False,
            'httpOnly': True,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        },
        {
            'name': 'ubid-main',
            'value': '135-7477912-2392604',
            'domain': '.goodreads.com',
            'path': '/',
            'secure': False,
            'httpOnly': True,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        },
        {
            'name': 'x-main',
            'value': 'hf5q7qpx3x0HgBwzCGe?XOExwXUrkUVkjz1Z15Z2ptg0JUWQ9klSPvPcAXY9G41Z',
            'domain': '.goodreads.com',
            'path': '/',
            'secure': False,
            'httpOnly': True,
            'expiry': int((datetime.now() + timedelta(days=365*2)).timestamp())
        }
    ]

    driver.get('https://www.goodreads.com/')
    print(f"{datetime.now()}: Navigating to Goodreads homepage for cookie loading.")
    time.sleep(2)
    driver.delete_all_cookies()
    print(f"{datetime.now()}: Cleared existing cookies.")
    for cookie in cookies:
        if 'expiry' in cookie:
            cookie['expiry'] = int(cookie['expiry'])
        driver.add_cookie(cookie)
    driver.refresh()
    print(f"{datetime.now()}: Loaded {len(cookies)} cookies and refreshed page.")
    time.sleep(3)

    if is_logged_in(driver):
        print(f"{datetime.now()}: Success: Cookie-based login successful!")
        return True
    else:
        print(f"{datetime.now()}: Cookie login failed. Falling back to auto-login if enabled.")
        if auto_login:
            return attempt_auto_login(driver)
        return False

def attempt_auto_login(driver):
    email = os.environ.get('GOODREADS_EMAIL')
    password = os.environ.get('GOODREADS_PASSWORD')
    if not email or not password:
        print(f"{datetime.now()}: ERROR: GOODREADS_EMAIL and GOODREADS_PASSWORD not set. Skipping auto-login.")
        return False
    print(f"{datetime.now()}: Attempting auto-login with provided credentials.")
    driver.get('https://www.goodreads.com/user/sign_in')
    time.sleep(3)
    driver.find_element(By.ID, 'user_email').send_keys(email)
    driver.find_element(By.ID, 'user_password').send_keys(password)
    driver.find_element(By.NAME, 'commit').click()
    time.sleep(5)
    return is_logged_in(driver)

def is_logged_in(driver):
    indicators = [
        (By.CLASS_NAME, 'siteHeader__personalMenu'),
        (By.XPATH, "//a[contains(@href, '/user/show/')]")
    ]
    for by, value in indicators:
        try:
            driver.find_element(by, value)
            return True
        except NoSuchElementException:
            continue
    return False

# Set up Selenium with anti-detection
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

driver = webdriver.Chrome(options=chrome_options)
load_cookies(driver, auto_login=True)  # Enable auto-login fallback if cookies fail

print("Cell 2: Login functions ready.")


=== Starting cookie/login process ===
2025-08-07 04:20:30.593550: Navigating to Goodreads homepage for cookie loading.
2025-08-07 04:20:33.017921: Cleared existing cookies.
2025-08-07 04:20:35.852029: Loaded 13 cookies and refreshed page.
2025-08-07 04:20:38.988625: Success: Cookie-based login successful!
Cell 2: Login functions ready.


# Cell 3: Main Scraping Functions and Execution

In [ ]:
def extract_abstract(soup):
    desc_elem = soup.find('div', class_='BookPageMetadataSection__description')
    if desc_elem:
        formatted_span = desc_elem.find('span', class_='Formatted')
        text = formatted_span.get_text(strip=True) if formatted_span else desc_elem.get_text(strip=True)
        print(f"{datetime.now()}: Scraped description: {text[:100]}...")  # Print first 100 chars for brevity
        return text
    print(f"{datetime.now()}: No description found.")
    return 'N/A'

def extract_core_identifiers(soup, url):
    print(f"{datetime.now()}: Extracting core identifiers.")
    title_elem = soup.find('h1', {'data-testid': 'bookTitle'}) or soup.find('h1', class_='gr-h1')
    title = title_elem.get_text(strip=True) if title_elem else 'N/A'
    print(f"{datetime.now()}: Scraped title: {title}")

    authors = [a.text.strip().replace('(Goodreads Author)', '').strip() for a in soup.find_all('span', {'data-testid': 'name'}) if 'Goodreads Author' not in a.text]
    author = authors[0] if authors else 'N/A'
    all_authors = '; '.join(authors) if len(authors) > 1 else author
    print(f"{datetime.now()}: Scraped author: {author}")
    print(f"{datetime.now()}: Scraped all authors: {all_authors}")

    book_id_match = re.search(r'(\d+)', url)
    book_id = book_id_match.group(1) if book_id_match else 'N/A'
    print(f"{datetime.now()}: Scraped book_id: {book_id}")

    return {
        'title': title,
        'author': author,
        'all_authors': all_authors,
        'book_id': book_id
    }

def extract_ratings_reviews(soup):
    print(f"{datetime.now()}: Extracting ratings and reviews metadata.")
    rating_elem = soup.find('div', class_='RatingStatistics__rating')
    average_rating = float(rating_elem.get_text(strip=True)) if rating_elem else 'N/A'
    print(f"{datetime.now()}: Scraped average_rating: {average_rating}")

    ratings_count_elem = soup.find('span', {'data-testid': 'ratingsCount'})
    num_ratings = int(re.search(r'(\d+[\d,]*)', ratings_count_elem.get_text(strip=True)).group(1).replace(',', '')) if ratings_count_elem else 'N/A'
    print(f"{datetime.now()}: Scraped num_ratings: {num_ratings}")

    reviews_count_elem = soup.find('span', {'data-testid': 'reviewsCount'})
    num_reviews = int(re.search(r'(\d+[\d,]*)', reviews_count_elem.get_text(strip=True)).group(1).replace(',', '')) if reviews_count_elem else 'N/A'
    print(f"{datetime.now()}: Scraped num_reviews: {num_reviews}")

    distribution = {}
    rating_bars = soup.find_all('div', class_='RatingStatistics__column')
    for i, bar in enumerate(rating_bars):
        aria_label = bar.get('aria-label', '')
        percentage_match = re.search(r'(\d+)%', aria_label)
        if percentage_match:
            distribution[f'{5-i}_star'] = int(percentage_match.group(1))
    print(f"{datetime.now()}: Scraped rating_distribution: {distribution}")

    return {
        'average_rating': average_rating,
        'num_ratings': num_ratings,
        'num_reviews': num_reviews,
        'rating_distribution': distribution
    }

def extract_publication_details(soup):
    print(f"{datetime.now()}: Extracting publication details.")
    pub_info_elem = soup.find('p', {'data-testid': 'publicationInfo'})
    pub_text = pub_info_elem.get_text(strip=True) if pub_info_elem else ''
    print(f"{datetime.now()}: Scraped pub_info: {pub_text}")

    year_match = re.search(r'(\d{4})', pub_text)
    pub_date = int(year_match.group(1)) if year_match else 'N/A'
    print(f"{datetime.now()}: Scraped pub_date: {pub_date}")

    publisher = pub_text.split('by ')[-1].strip() if 'by ' in pub_text else 'N/A'
    print(f"{datetime.now()}: Scraped publisher: {publisher}")

    format_elem = soup.find('p', {'data-testid': 'pagesFormat'})
    format_text = format_elem.get_text(strip=True) if format_elem else ''
    print(f"{datetime.now()}: Scraped format_info: {format_text}")

    pages_match = re.search(r'(\d+) pages', format_text)
    page_count = int(pages_match.group(1)) if pages_match else 'N/A'
    print(f"{datetime.now()}: Scraped page_count: {page_count}")

    return {
        'pub_info': pub_text,
        'pub_date': pub_date,
        'publisher': publisher,
        'isbn': soup.find('span', itemprop='isbn').text if soup.find('span', itemprop='isbn') else 'N/A',
        'format_info': format_text,
        'page_count': page_count,
        'language': 'English'  # Default or parse if available
    }

def extract_genres_shelves(soup):
    print(f"{datetime.now()}: Extracting genres and shelves.")
    genres = []
    genre_section = soup.find('div', class_='BookPageMetadataSection__genres')
    if genre_section:
        genre_links = genre_section.find_all('a', class_='Button')
        genres = [link.text.strip() for link in genre_links[:10]]  # Top 10
    primary_genre = genres[0] if genres else 'N/A'
    genres_str = '; '.join(genres) if genres else 'N/A'
    print(f"{datetime.now()}: Scraped genres: {genres_str}")
    print(f"{datetime.now()}: Scraped primary_genre: {primary_genre}")

    return {
        'genres': genres_str,
        'primary_genre': primary_genre
    }

def extract_characters_and_places(soup):
    print(f"{datetime.now()}: Extracting characters and places.")
    characters = [c.text.strip() for c in soup.select('div[data-testid="characters"] a')]
    places = [p.text.strip() for p in soup.select('div[data-testid="places"] a')]
    characters_str = '; '.join(characters) if characters else 'N/A'
    places_str = '; '.join(places) if places else 'N/A'
    print(f"{datetime.now()}: Scraped characters: {characters_str}")
    print(f"{datetime.now()}: Scraped places: {places_str}")

    return {
        'characters': characters_str,
        'places': places_str
    }

def collect_reviews(driver, book_id, max_reviews=100):
    reviews_url = f"https://www.goodreads.com/book/show/{book_id}/reviews?review_filters=has_user_rating"
    print(f"{datetime.now()}: Navigating to reviews URL: {reviews_url}")
    driver.get(reviews_url)
    time.sleep(random.uniform(2, 4))

    reviews = []
    loaded = 0
    pbar = tqdm(total=max_reviews, desc="Collecting reviews")
    while loaded < max_reviews:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        review_elements = soup.find_all('section', class_='ReviewCard')[loaded:]
        for rev in review_elements:
            rating = rev.find('span', class_='RatingStars')['aria-label'] if rev.find('span', class_='RatingStars') else 'N/A'
            text = rev.find('span', class_='ReviewText').text.strip() if rev.find('span', class_='ReviewText') else 'N/A'
            reviews.append({'rating': rating, 'text': text})
            loaded += 1
            pbar.update(1)
            if loaded >= max_reviews:
                break

        try:
            load_more = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Load More')]")))
            print(f"{datetime.now()}: Clicking 'Load More' (loaded so far: {loaded})")
            driver.execute_script("arguments[0].click();", load_more)
            time.sleep(random.uniform(2, 4))
        except TimeoutException:
            print(f"{datetime.now()}: No more 'Load More' button or timeout. Stopping review collection.")
            break
    pbar.close()

    # Print scraped reviews summary and sample
    print(f"{datetime.now()}: Collected {len(reviews)} reviews.")
    if reviews:
        print(f"{datetime.now()}: Sample of first 5 reviews:")
        for idx, rev in enumerate(reviews[:5]):
            print(f"Review {idx+1}: Rating = {rev['rating']}, Text = {rev['text'][:100]}...")  # First 100 chars for brevity

    return reviews

def enrich_book_metadata(row):
    start_time = datetime.now()
    print(f"\n{start_time}: Starting metadata enrichment for book: {row['title']} (URL: {row['url']})")

    driver.get(row['url'])
    time.sleep(random.uniform(2, 4))

    # Expand sections
    print(f"{datetime.now()}: Expanding sections...")
    expand_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), 'more') or contains(text(), 'More') or @aria-label='Show all']")
    for btn in expand_buttons:
        try:
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(1)
        except:
            print(f"{datetime.now()}: Failed to click an expand button (continuing).")

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    metadata = extract_core_identifiers(soup, row['url'])
    metadata.update(extract_ratings_reviews(soup))
    metadata.update(extract_publication_details(soup))
    metadata.update(extract_genres_shelves(soup))
    metadata.update(extract_characters_and_places(soup))
    metadata['description'] = extract_abstract(soup)

    duration = (datetime.now() - start_time).total_seconds()
    print(f"{datetime.now()}: Metadata enrichment complete for {row['title']} (took {duration:.2f} seconds).")
    return pd.Series(metadata)

# Load CSV
csv_path = '/content/drive/MyDrive/Romantic_Extened_Dataset_Scraping/romantic_books_final_scraped.csv'
df = pd.read_csv(csv_path)
print(f"{datetime.now()}: Loaded DataFrame with {len(df)} rows.")

# For testing: Process only first 3 rows (comment out for full dataset)
df = df.head(3)
print(f"{datetime.now()}: Testing on sample of {len(df)} books (first 3 rows).")

# Enrich metadata with progress bar
enriched_data = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Enriching metadata for books"):
    try:
        enriched_data.append(enrich_book_metadata(row))
    except Exception as e:
        print(f"{datetime.now()}: Error enriching metadata for {row['title']}: {e}. Skipping.")
df = pd.concat([df] + enriched_data, axis=1)

# Collect reviews for each book with progress
for i, row in tqdm(df.iterrows(), total=len(df), desc="Collecting reviews for books"):
    try:
        start_time = datetime.now()
        print(f"\n{start_time}: Starting review collection for book: {row['title']} (ID: {row['book_id']})")
        df.at[i, 'reviews'] = json.dumps(collect_reviews(driver, row['book_id']))
        duration = (datetime.now() - start_time).total_seconds()
        print(f"{datetime.now()}: Review collection complete for {row['title']} (took {duration:.2f} seconds).")
    except Exception as e:
        print(f"{datetime.now()}: Error collecting reviews for {row['title']}: {e}. Skipping.")

# Save enriched CSV
enriched_path = '/content/drive/MyDrive/Romantic_Extened_Dataset_Scraping/romantic_books_enriched.csv'
df.to_csv(enriched_path, index=False)
print(f"{datetime.now()}: Enriched CSV saved to {enriched_path}")

driver.quit()
print(f"{datetime.now()}: Cell 3: Enrichment complete.")

2025-08-07 04:20:39.033697: Loaded DataFrame with 2860 rows.
2025-08-07 04:20:39.034138: Testing on sample of 3 books (first 3 rows).


Enriching metadata for books:   0%|          | 0/3 [00:00<?, ?it/s]


2025-08-07 04:20:39.040829: Starting metadata enrichment for book: The Love Hypothesis (Paperback) (URL: https://www.goodreads.com/book/show/56732449-the-love-hypothesis)
